In [1]:
import librosa
import os as os
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
import numpy as np
from tqdm import tqdm

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
DATA_PATH = "./data/"


# Input: Folder Path
# Output: Tuple (Label, Indices of the labels, one-hot encoded labels)
def get_labels(path=DATA_PATH):
    labels = os.listdir(path)
    label_indices = np.arange(0, len(labels))
    return labels, label_indices, to_categorical(label_indices)


# Handy function to convert wav2mfcc
def wav2mfcc(file_path, max_len=11):
    wave, sr = librosa.load(file_path, mono=True, sr=None)
    wave = wave[::3]
    mfcc = librosa.feature.mfcc(wave, sr=16000)

    # If maximum length exceeds mfcc lengths then pad the remaining ones
    if (max_len > mfcc.shape[1]):
        pad_width = max_len - mfcc.shape[1]
        mfcc = np.pad(mfcc, pad_width=((0, 0), (0, pad_width)), mode='constant')

    # Else cutoff the remaining parts
    else:
        mfcc = mfcc[:, :max_len]
    return mfcc


def save_data_to_array(path=DATA_PATH, max_len=11):
    labels, _, _ = get_labels(path)

    for label in labels:
        # Init mfcc vectors
        mfcc_vectors = []

        wavfiles = [path + label + '/' + wavfile for wavfile in os.listdir(path + '/' + label)]
        for wavfile in tqdm(wavfiles, "Saving vectors of label - '{}'".format(label)):
            mfcc = wav2mfcc(wavfile, max_len=max_len)
            mfcc_vectors.append(mfcc)
        np.save(label + '.npy', mfcc_vectors)


def get_train_test(split_ratio=0.6, random_state=42):
    # Get available labels
    labels, indices, _ = get_labels(DATA_PATH)

    # Getting first arrays
    X = np.load(labels[0] + '.npy')
    y = np.zeros(X.shape[0])

    # Append all of the dataset into one single array, same goes for y
    for i, label in enumerate(labels[1:]):
        x = np.load(label + '.npy')
        X = np.vstack((X, x))
        y = np.append(y, np.full(x.shape[0], fill_value= (i + 1)))

    assert X.shape[0] == len(y)

    return train_test_split(X, y, test_size= (1 - split_ratio), random_state=random_state, shuffle=True)



def prepare_dataset(path=DATA_PATH):
    labels, _, _ = get_labels(path)
    data = {}
    for label in labels:
        data[label] = {}
        data[label]['path'] = [path  + label + '/' + wavfile for wavfile in os.listdir(path + '/' + label)]

        vectors = []

        for wavfile in data[label]['path']:
            wave, sr = librosa.load(wavfile, mono=True, sr=None)
            # Downsampling
            wave = wave[::3]
            mfcc = librosa.feature.mfcc(wave, sr=16000)
            vectors.append(mfcc)

        data[label]['mfcc'] = vectors

    return data


def load_dataset(path=DATA_PATH):
    data = prepare_dataset(path)

    dataset = []

    for key in data:
        for mfcc in data[key]['mfcc']:
            dataset.append((key, mfcc))

    return dataset[:100]

In [3]:
get_train_test()
print('get_train_test')
prepare_dataset()
print('prepare_dataset')
load_dataset
print('load_dataset')

get_train_test
prepare_dataset
load_dataset


In [4]:
%load_ext autoreload
%autoreload 2


import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.utils import to_categorical

# Second dimension of the feature is dim2
feature_dim_2 = 11

# Save data to array file first
save_data_to_array(max_len=feature_dim_2)

# # Loading train set and test set
X_train, X_test, y_train, y_test = get_train_test()

# # Feature dimension
feature_dim_1 = 20
channel = 1
epochs = 50
batch_size = 100
verbose = 1
num_classes = 2

# Reshaping to perform 2D convolution
X_train = X_train.reshape(X_train.shape[0], feature_dim_1, feature_dim_2, channel)
X_test = X_test.reshape(X_test.shape[0], feature_dim_1, feature_dim_2, channel)

y_train_hot = to_categorical(y_train)
y_test_hot = to_categorical(y_test)

Saving vectors of label - 'dog': 100%|█████████████████████████████████████████████████████████████████████| 1746/1746 [00:09<00:00, 181.60it/s]


In [5]:
def get_model():
    model = Sequential()
    model.add(Conv2D(32, kernel_size=(2, 2), activation='relu', input_shape=(feature_dim_1, feature_dim_2, channel)))
    model.add(Conv2D(48, kernel_size=(2, 2), activation='relu'))
    model.add(Conv2D(120, kernel_size=(2, 2), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.25))
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.4))
    model.add(Dense(num_classes, activation='softmax'))
    model.compile(loss=keras.losses.categorical_crossentropy,
                  optimizer=keras.optimizers.Adadelta(),
                  metrics=['accuracy'])
    return model

# Predicts one sample
def predict(filepath, model):
    sample = wav2mfcc(filepath)
    sample_reshaped = sample.reshape(1, feature_dim_1, feature_dim_2, channel)
    return get_labels()[0][
            np.argmax(model.predict(sample_reshaped))
    ]

In [6]:
model = get_model()
model.fit(X_train, y_train_hot,
          batch_size=batch_size, 
          epochs=epochs, 
          verbose=verbose
         )

Epoch 1/50
2087/2087 [==============================] - ETA: 1:12 - loss: 3.3642 - acc: 0.530 - ETA: 35s - loss: 4.7143 - acc: 0.535 - ETA: 23s - loss: 3.8761 - acc: 0.55 - ETA: 17s - loss: 4.0298 - acc: 0.52 - ETA: 13s - loss: 3.5926 - acc: 0.53 - ETA: 10s - loss: 3.2263 - acc: 0.52 - ETA: 8s - loss: 2.9025 - acc: 0.5429 - ETA: 7s - loss: 2.6516 - acc: 0.550 - ETA: 6s - loss: 2.4587 - acc: 0.553 - ETA: 5s - loss: 2.2850 - acc: 0.563 - ETA: 4s - loss: 2.1395 - acc: 0.575 - ETA: 3s - loss: 2.0314 - acc: 0.576 - ETA: 3s - loss: 1.9253 - acc: 0.576 - ETA: 2s - loss: 1.8319 - acc: 0.584 - ETA: 2s - loss: 1.7458 - acc: 0.594 - ETA: 1s - loss: 1.6745 - acc: 0.601 - ETA: 1s - loss: 1.6113 - acc: 0.607 - ETA: 0s - loss: 1.5534 - acc: 0.614 - ETA: 0s - loss: 1.5007 - acc: 0.619 - ETA: 0s - loss: 1.4536 - acc: 0.624 - 6s 3ms/step - loss: 1.4156 - acc: 0.6301
Epoch 2/50
2087/2087 [==============================] - ETA: 2s - loss: 0.5213 - acc: 0.750 - ETA: 2s - loss: 0.4888 - acc: 0.765 - ETA: 2s

2087/2087 [==============================] - ETA: 2s - loss: 0.0083 - acc: 1.000 - ETA: 2s - loss: 0.0064 - acc: 1.000 - ETA: 2s - loss: 0.0099 - acc: 1.000 - ETA: 2s - loss: 0.0092 - acc: 1.000 - ETA: 2s - loss: 0.0111 - acc: 0.998 - ETA: 1s - loss: 0.0188 - acc: 0.995 - ETA: 1s - loss: 0.0165 - acc: 0.995 - ETA: 1s - loss: 0.0154 - acc: 0.996 - ETA: 1s - loss: 0.0144 - acc: 0.996 - ETA: 1s - loss: 0.0131 - acc: 0.997 - ETA: 1s - loss: 0.0133 - acc: 0.996 - ETA: 1s - loss: 0.0143 - acc: 0.995 - ETA: 1s - loss: 0.0158 - acc: 0.995 - ETA: 0s - loss: 0.0155 - acc: 0.995 - ETA: 0s - loss: 0.0153 - acc: 0.995 - ETA: 0s - loss: 0.0150 - acc: 0.995 - ETA: 0s - loss: 0.0150 - acc: 0.995 - ETA: 0s - loss: 0.0147 - acc: 0.996 - ETA: 0s - loss: 0.0143 - acc: 0.996 - ETA: 0s - loss: 0.0138 - acc: 0.996 - 3s 1ms/step - loss: 0.0148 - acc: 0.9957
Epoch 20/50
2087/2087 [==============================] - ETA: 2s - loss: 0.0468 - acc: 0.980 - ETA: 2s - loss: 0.0266 - acc: 0.990 - ETA: 2s - loss: 0.020

2087/2087 [==============================] - ETA: 2s - loss: 0.0036 - acc: 1.000 - ETA: 2s - loss: 0.0023 - acc: 1.000 - ETA: 2s - loss: 0.0067 - acc: 0.996 - ETA: 2s - loss: 0.0056 - acc: 0.997 - ETA: 2s - loss: 0.0047 - acc: 0.998 - ETA: 2s - loss: 0.0042 - acc: 0.998 - ETA: 2s - loss: 0.0038 - acc: 0.998 - ETA: 1s - loss: 0.0033 - acc: 0.998 - ETA: 1s - loss: 0.0030 - acc: 0.998 - ETA: 1s - loss: 0.0030 - acc: 0.999 - ETA: 1s - loss: 0.0028 - acc: 0.999 - ETA: 1s - loss: 0.0026 - acc: 0.999 - ETA: 1s - loss: 0.0026 - acc: 0.999 - ETA: 1s - loss: 0.0025 - acc: 0.999 - ETA: 0s - loss: 0.0031 - acc: 0.998 - ETA: 0s - loss: 0.0029 - acc: 0.998 - ETA: 0s - loss: 0.0030 - acc: 0.998 - ETA: 0s - loss: 0.0029 - acc: 0.998 - ETA: 0s - loss: 0.0028 - acc: 0.998 - ETA: 0s - loss: 0.0028 - acc: 0.999 - 3s 1ms/step - loss: 0.0027 - acc: 0.9990
Epoch 38/50
2087/2087 [==============================] - ETA: 2s - loss: 3.4432e-04 - acc: 1.000 - ETA: 2s - loss: 2.4517e-04 - acc: 1.000 - ETA: 2s - los

In [7]:
print(predict('./data/cat/1a9afd33_nohash_0.wav', model=model))

cat


In [8]:
test_loss, test_acc = model.evaluate(X_test, y_test_hot)

1392/1392 [==============================] - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - 1s 388us/step


In [9]:
test_acc

0.9712643678160919

In [1]:
from sklearn.metrics import roc_auc_score